<a href="https://colab.research.google.com/github/jmhuer/shift_invariant_dictionary_learning/blob/main/WTA_drum_encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install skorch
!pip install tqdm

     |████████████████████████████████| 133kB 8.2MB/s 


# Plot Utils 

In [2]:
import plotly.graph_objects as graph
def plot(all_history:list, title:str, log = False):
    """
    input:
        all_history: list of dicts to plot
    ret:
        None: show plotly fig
    """
    #symbol_sequence= ['circle-open', 'circle', 'circle-open-dot', 'square']
    fig = graph.Figure(layout = graph.Layout(title=graph.layout.Title(text=title))) 
    for i in range(len(all_history)):
        fig.add_trace(graph.Scatter(x = all_history[i]["x"], 
                                    y = all_history[i]["y"],
                                    name = all_history[i]["legend"],
                                    mode='markers',
                                    marker_size=5,
                                    marker_symbol=all_history[i]["marker_symbol"])) 
    if log: fig.update_xaxes(type="log")
    fig.show()

# Generate some drum synthetic data similar to symbolic music

In [1]:
# lets make the data 400 in lenghth to match autoencoder imlementation 
#lets have 10 different sections each 40 in lenght -- ideal conditions 

import numpy as np

print("beat patter dictionary ")
## basic patters
downbeat    = np.array([-2 if i % 4 == 0 else 0 for i in range(0,40)]) 
downbeat2x  = np.array([-2 if i % 2 == 0 else 0 for i in range(0,40)]) 
high_hats   = np.array([2 if i % 4 == 1 else 0 for i in range(0,40)]) 
high_hats2x = np.array([2 if i % 2 == 1 else 0 for i in range(0,40)]) 
tom_drum    = np.array([-1 if i % 4 == 2 else 0 for i in range(0,40)]) 

## combine basic patters
comb1 = downbeat   + high_hats 
comb2 = downbeat   + high_hats2x
comb3 = downbeat2x + high_hats2x
comb4 = downbeat   + high_hats2x  + tom_drum
comb5 = downbeat2x + high_hats


print(list(downbeat))
print(list(downbeat2x))
print(list(high_hats))
print(list(high_hats2x))
print(list(tom_drum), "\n")

print(list(comb1))
print(list(comb2))
print(list(comb3))
print(list(comb4))
print(list(comb5), "\n")


##here is the list we will permute
musical_sections = [downbeat, downbeat2x, high_hats, high_hats2x, tom_drum, comb1, comb2, comb3, comb4, comb5]

from itertools import permutations

synth_data = np.array(list(permutations(musical_sections))[0:10000]) ##okay but eventually you might need to trucate the permutations

#reshape and unravel
(a,b,c) = synth_data.shape
synth_data = synth_data.reshape((a, b*c))
# synth_data = synth_data[:,None,:] #add channel dim
print("size of dataset ", synth_data.shape)

print("Example datapoint:\n ", synth_data[0])


beat patter dictionary 
[-2, 0, 0, 0, -2, 0, 0, 0, -2, 0, 0, 0, -2, 0, 0, 0, -2, 0, 0, 0, -2, 0, 0, 0, -2, 0, 0, 0, -2, 0, 0, 0, -2, 0, 0, 0, -2, 0, 0, 0]
[-2, 0, -2, 0, -2, 0, -2, 0, -2, 0, -2, 0, -2, 0, -2, 0, -2, 0, -2, 0, -2, 0, -2, 0, -2, 0, -2, 0, -2, 0, -2, 0, -2, 0, -2, 0, -2, 0, -2, 0]
[0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0]
[0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2]
[0, 0, -1, 0, 0, 0, -1, 0, 0, 0, -1, 0, 0, 0, -1, 0, 0, 0, -1, 0, 0, 0, -1, 0, 0, 0, -1, 0, 0, 0, -1, 0, 0, 0, -1, 0, 0, 0, -1, 0] 

[-2, 2, 0, 0, -2, 2, 0, 0, -2, 2, 0, 0, -2, 2, 0, 0, -2, 2, 0, 0, -2, 2, 0, 0, -2, 2, 0, 0, -2, 2, 0, 0, -2, 2, 0, 0, -2, 2, 0, 0]
[-2, 2, 0, 2, -2, 2, 0, 2, -2, 2, 0, 2, -2, 2, 0, 2, -2, 2, 0, 2, -2, 2, 0, 2, -2, 2, 0, 2, -2, 2, 0, 2, -2, 2, 0, 2, -2, 2, 0, 2]
[-2, 2, -2, 2, -2, 2, -2, 2, -2, 2, -2, 2, -2, 2, -2, 2, -2, 2, -2,

# Auto Econder

In [3]:
import torch
import torch.optim as optim
torch.manual_seed(42)

class autoencoder(torch.nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = torch.nn.Conv1d(in_channels=1, out_channels=10, kernel_size=40, padding=0, bias=False, stride=40)
        self.decoder = torch.nn.ConvTranspose1d(in_channels=10, out_channels=1, kernel_size=40, padding=0, bias=False, stride=40)
        # self.encoder.weight.data.fill_(4)
        # self.encoder.weight.data.fill_(0.1)
    def get_kernels(self):
        return self.encoder.weight.data[:,0,:]
    def feature_map(self, x):
        code = self.encoder(x)
        return code
    def forward(self, x):
        code = self.encoder(x)
        reconstruct = self.decoder(code)
        return reconstruct


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Using device: ", device)

model = autoencoder().to(device)
inputs = torch.tensor(synth_data).float().to(device)
out = model(inputs)

print("Input size: ", inputs.size())
print("Output size: ", out.size(), "\n")


loss_fn = torch.nn.L1Loss().to(device)
optimizer = optim.SGD(model.parameters(), lr=.05, weight_decay = 0.00001, momentum=0.05) ##this has weight decay just like you implemented
epochs = 1000
history = {"loss": []}
for i in range(epochs):
  optimizer.zero_grad()
  output = model(inputs)
  # num_weights = float(model.encoder.weight.ravel().size()[0])
  # my_loss = (model.encoder.weight.ravel() < 0).sum(dim=0) / num_weights #count number of negative values in kernels
  # print("my loss ", float(my_loss))
  # print("total encoder ", num_weights)

  loss = loss_fn(output, inputs)
  loss.backward()
  optimizer.step()
  history["loss"].append(float(loss))
  if i % 10 == 0: 
      print("Epoch : {} \t Loss : {} \t ".format(i, round(float(loss),4)))
      print("\nneg encoder ", float((model.encoder.weight.ravel() < 0).sum(dim=0)))





Using device:  cuda


RuntimeError: ignored

In [ ]:
#loss

#perfect plot
loss_plot = {"legend": "original", 
             "x": list(range(0,len(history["loss"]))), 
             "y": history["loss"],
             "marker_symbol": 'star'}
     

plot([loss_plot], "loss")

In [ ]:
data = list(model.get_kernels()[4].tolist())

recunstruct = model(torch.tensor(inputs[0:1,:,:]).float())
# print(recunstruct.detach().numpy()[0,0,:])

#perfect plot
original_plot = {"legend": "original", 
                 "x": list(range(0,400)), 
                 "y": synth_data[0,0,:],
                 "marker_symbol": 'line-ne-open'}
                  
recunstruct_plot = {"legend": "reconstruct", 
             "x": list(range(0,400)), 
             "y": recunstruct.cpu().detach().numpy()[0,0,:],
             "marker_symbol": 'star'}      

plot([recunstruct_plot, original_plot], "Signal Comparison")

# see kernels

In [ ]:
kernel1 = model.get_kernels().cpu().numpy()[1].tolist()
kernel0 = model.get_kernels().cpu().numpy()[0].tolist()
kernel6 = model.get_kernels().cpu().numpy()[6].tolist()


kernels6_plot  = {"legend": "original", 
                 "x": list(range(0,40)), 
                 "y": kernel6,
                 "marker_symbol": 'triangle-up-open'}

#perfect plot
kernels1_plot  = {"legend": "original", 
                 "x": list(range(0,40)), 
                 "y": kernel1,
                 "marker_symbol": 'star'}

plot([kernels5_plot], "kernels_plot")

# now plot one feature map

In [ ]:

feature = list(model.feature_map(torch.tensor(inputs[0:1,:,:]).float()).detach().cpu().numpy()[0,0,:])
print(feature)

#feature plot
feature_plot  = {"legend": "feature", 
                 "x": list(range(0,40)), 
                 "y": feature,
                 "marker_symbol": 'star'}

plot([feature_plot], "feature_plot")

[0.18243606, -0.544937, 4.0979056, 3.5438564, -2.1821191, 4.280341, 3.7262921, 2.9989192, 1.5441734, 3.5529683]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



#okzy for the above eample find highiest match section and plot kernel and section together

In [ ]:
kernel5 = (model.get_kernels().cpu().numpy()*5 + 1)[5].tolist()
# print(kernel5)
# print(kernel5*6)


kernels5_plot  = {"legend": "kernels5_plot", 
                 "x": list(range(0,40)), 
                 "y": kernel5,
                 "marker_symbol": 'triangle-up-open'}

section5_plot  = {"legend": "section5_plot", 
                 "x": list(range(0,40)), 
                 "y": inputs[0:1,:,:].cpu().numpy().ravel()[5*40:6*40],
                 "marker_symbol": 'circle'}


print(inputs[0:1,:,:].cpu().numpy().ravel()[6*40:7*40])

plot([kernels5_plot,section5_plot], "kernel and features")


[1. 2. 0. 2. 1. 2. 0. 2. 1. 2. 0. 2. 1. 2. 0. 2. 1. 2. 0. 2. 1. 2. 0. 2.
 1. 2. 0. 2. 1. 2. 0. 2. 1. 2. 0. 2. 1. 2. 0. 2.]


# WTA implemetnatin


In [4]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, transforms
import torchvision




class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.shape[0], -1)


class SparsifyBase(nn.Module):
    def __init__(self, sparse_ratio=0.5):
        super(SparsifyBase, self).__init__()
        self.sr = sparse_ratio
        self.preact = None
        self.act = None

    def get_activation(self):
        def hook(model, input, output):
            self.preact = input[0].cpu().detach().clone()
            self.act = output.cpu().detach().clone()
        return hook
    
    def record_activation(self):
        self.register_forward_hook(self.get_activation())


class Sparsify1D(SparsifyBase):
    def __init__(self, sparse_ratio=0.5):
        super(Sparsify1D, self).__init__()
        self.sr = sparse_ratio

    def forward(self, x):
        k = int(self.sr*x.shape[1])
        topval = x.topk(k, dim=1)[0][:, -1]
        topval = topval.expand(x.shape[1], x.shape[0]).permute(1,0)
        comp = (x>=topval).to(x)
        return comp*x

class Sparsify1D_kactive(SparsifyBase):
    def __init__(self, k=1):
        super(Sparsify1D_kactive, self).__init__()
        self.k = k
    
    def forward(self, x):
        k = self.k
        topval = x.topk(k, dim=1)[0][-1,:]
        # topval = topval.expand(x.shape[1], x.shape[0]).permute(1,0)
        comp = (x>=topval).to(x)
        return comp*x 

In [52]:
from torch.utils.data import Dataset, DataLoader

##data loading 

inputs = torch.tensor(synth_data).float().to(device)

print("Input size: ", inputs.size())
# print("Output size: ", out.size(), "\n")


class TimeseriesDataset(torch.utils.data.Dataset):   
    def __init__(self, X, y, seq_len):
        self.X = X
        self.y = y
        self.seq_len = seq_len
    def __len__(self):
        return self.X.__len__() - (self.seq_len-1)
    def __getitem__(self, index):
        # return (self.X[index:index+self.seq_len], self.y[index+self.seq_len-1])
        return self.X[index:index+1]


# for i, d in enumerate(train_loader):
#     print(i, d.size())

Input size:  torch.Size([10000, 400])


In [ ]:
import torch
import torch.optim as optim
torch.manual_seed(42)

class autoencoder(torch.nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.relu1 = nn.ReLU(inplace=False)
        self.wta = Sparsify1D_kactive()
        self.encoder = torch.nn.Conv1d(in_channels=1, out_channels=10, kernel_size=40, padding=0, bias=False, stride=40)
        self.decoder = torch.nn.ConvTranspose1d(in_channels=10, out_channels=1, kernel_size=40, padding=0, bias=False, stride=40)
        # self.encoder.weight.data.fill_(-.4)
        self.decoder.weight.data.fill_(0)
    def get_kernels(self):
        return self.encoder.weight.data[:,0,:]
    def feature_map(self, x):
        code = self.encoder(x)
        return code
    def forward(self, x):
        code =  self.wta(self.encoder(x))
        reconstruct = self.decoder(code)
        return reconstruct


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Using device: ", device)


train_dataset = TimeseriesDataset(inputs, None, seq_len=400)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 10, shuffle = True)
model = autoencoder().to(device)


loss_fn = torch.nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=.05, weight_decay = 0.00001, momentum=0.05) ##this has weight decay just like you implemented
epochs = 1000
history = {"loss": []}
for i in range(epochs):
  for j, inputs in enumerate(train_loader):
    inputs = inputs.to(device)
    optimizer.zero_grad()
    output = model(inputs)
    # num_weights = float(model.encoder.weight.ravel().size()[0])
    # my_loss = (model.encoder.weight.ravel() < 0).sum(dim=0) / num_weights #count number of negative values in kernels
    # print("my loss ", float(my_loss))
    # print("total encoder ", num_weights)

    loss = loss_fn(output, inputs)
    loss.backward()
    optimizer.step()
    history["loss"].append(float(loss))
    if i % 10 == 0: 
        print("Epoch : {} \t Loss : {} \t ".format(i, round(float(loss),4)))
        print("\nneg encoder ", float((model.encoder.weight.ravel() < 0).sum(dim=0)))





Streaming output truncated to the last 5000 lines.

neg encoder  203.0
Epoch : 110 	 Loss : 0.8136 	 

neg encoder  203.0
Epoch : 110 	 Loss : 0.879 	 

neg encoder  203.0
Epoch : 110 	 Loss : 0.9214 	 

neg encoder  203.0
Epoch : 110 	 Loss : 0.8433 	 

neg encoder  203.0
Epoch : 110 	 Loss : 0.8063 	 

neg encoder  203.0
Epoch : 110 	 Loss : 0.8606 	 

neg encoder  203.0
Epoch : 110 	 Loss : 0.7503 	 

neg encoder  203.0
Epoch : 110 	 Loss : 0.8188 	 

neg encoder  203.0
Epoch : 110 	 Loss : 0.8338 	 

neg encoder  203.0
Epoch : 110 	 Loss : 0.8259 	 

neg encoder  203.0
Epoch : 110 	 Loss : 0.9065 	 

neg encoder  203.0
Epoch : 110 	 Loss : 0.7411 	 

neg encoder  203.0
Epoch : 110 	 Loss : 0.931 	 

neg encoder  203.0
Epoch : 110 	 Loss : 0.8654 	 

neg encoder  203.0
Epoch : 110 	 Loss : 0.8279 	 

neg encoder  203.0
Epoch : 110 	 Loss : 0.8846 	 

neg encoder  203.0
Epoch : 110 	 Loss : 0.7717 	 

neg encoder  203.0
Epoch : 110 	 Loss : 0.8998 	 

neg encoder  203.0
Epoch : 110 	

In [50]:
data = list(model.get_kernels()[4].tolist())

recunstruct = model(torch.tensor(inputs[0:1,:,:]).float())
# print(recunstruct.detach().numpy()[0,0,:])

#perfect plot
original_plot = {"legend": "original", 
                 "x": list(range(0,400)), 
                 "y": synth_data[0,:],
                 "marker_symbol": 'line-ne-open'}
                  
recunstruct_plot = {"legend": "reconstruct", 
             "x": list(range(0,400)), 
             "y": recunstruct.cpu().detach().numpy()[0,0,:],
             "marker_symbol": 'star'}      

plot([recunstruct_plot, original_plot], "Signal Comparison")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

